In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging

for handler in logging.root.handlers:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO)

In [3]:
# Set project root
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
PROJECT_ROOT_LS = [p.name for p in PROJECT_ROOT.iterdir()]
assert "featurologists" in PROJECT_ROOT_LS, f"Not a project root? {PROJECT_ROOT}, pwd: {Path().resolve()}"

In [4]:
import os

K8S_FEAST_NS = os.environ.get('K8S_FEAST_NS', 'feast-dev')
print(f'K8S_FEAST_NS={K8S_FEAST_NS}')

K8S_FEAST_NS=feast-dev


In [8]:
from featurologists.utils import kubectl_port_forward

In [9]:
port_forward_processes = []

In [10]:
feast_core_process, feast_core_port = kubectl_port_forward(K8S_FEAST_NS, 'service/feast-release-feast-core', 6565)

port_forward_processes.append(feast_core_process)

pid: 341319
Port-forward process for 'service/feast-release-feast-core' seems to be working: check 'localhost:34971'


In [11]:
import os

from feast import Client, Feature, Entity, ValueType, FeatureTable
from feast.data_source import FileSource, KafkaSource
from feast.data_format import ParquetFormat, AvroFormat

In [12]:
client = Client(core_url=f"localhost:{feast_core_port}") #, serving_url='localhost:8092')
client.list_projects()

/plain/github/opensource/Featurologists/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['default']

In [13]:
DATA_FEAST_PATH = PROJECT_ROOT / 'data-feast'

offline_clusters_location = f'file://{DATA_FEAST_PATH}'

In [16]:
! head -n2 {PROJECT_ROOT}/data/output/cancellation_prediction/offline_preprocessed.csv
# Quantity,UnitPrice,Country,InvoiceDate_year,InvoiceDate_month,InvoiceDate_day,InvoiceDate_hour,InvoiceDate_minute,InvoiceDate_second,IsCancelled

Quantity,UnitPrice,Country,InvoiceDate_year,InvoiceDate_month,InvoiceDate_day,InvoiceDate_hour,InvoiceDate_minute,InvoiceDate_second,IsCancelled
6,2.55,34,2010,12,1,8,26,0,0


In [13]:
invoice_id = Entity(name="invoice_id", description="Invoice ID", value_type=ValueType.INT64)

offline_clusters_source_uri = os.path.join(offline_clusters_location, "offline_preprocessed_csv")

customer_clusters = FeatureTable(
    name = "invoices",
    entities = ["invoice_id"],
    features = [
        Feature("Quantity", ValueType.INT64),
        Feature("UnitPrice", ValueType.FLOAT),
        Feature("Country", ValueType.INT64),
        Feature("InvoiceDate_year", ValueType.INT64),
        Feature("InvoiceDate_month", ValueType.INT64),
        Feature("InvoiceDate_day", ValueType.INT64),
        Feature("InvoiceDate_hour", ValueType.INT64),
        Feature("InvoiceDate_minute", ValueType.INT64),
        Feature("InvoiceDate_second", ValueType.INT64),
        Feature("IsCancelled", ValueType.INT64),        
    ],
    batch_source=FileSource(
        event_timestamp_column="",
        created_timestamp_column="created",
        file_format=ParquetFormat(),
        file_url=driver_statistics_source_uri,
        date_partition_column="date"
    )
)

SyntaxError: invalid syntax (<ipython-input-13-b9f61324c485>, line 27)

In [68]:
for p in port_forward_processes:
    p.kill()

/plain/github/opensource/Featurologists/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
